In [62]:
import cvxpy as cp
import time
import clarabel
import torch
from zap.admm import ADMMSolver
from zap.conic.cone_bridge import ConeBridge
import scipy.sparse as sp
from zap.conic.cone_utils import generate_max_flow_problem, is_valid_network


In [63]:
## Create a large problem that is valid 
n = 1000
seed = 42
valid_source_sink_path = False

while (not valid_source_sink_path):
    problem, adj, inc = generate_max_flow_problem(n, seed)
    valid_source_sink_path = is_valid_network(adj)
    if (not valid_source_sink_path):
        seed += 1
nnz = float(adj.nnz)
total_possible_edges = float(adj.shape[0]*(adj.shape[0] - 1))
density = nnz/total_possible_edges
sparsity = 1 - density
num_variables = sum(np.prod(var.shape) for var in problem.variables())
num_constraints = sum(constraint.size for constraint in problem.constraints)
print(f'Generated a valid network with {n} nodes using seed {seed}')
print(f"Actual Number of Edges: {nnz}")
print(f"Total Possible Edges: {total_possible_edges}")
print(f"Graph sparsity: {sparsity}")
print(f"Graph density: {density}")
print(f"Number of Variables: {num_variables}")
print(f"Number of Constraints: {num_constraints}")

Generated a valid network with 1000 nodes using seed 42
Actual Number of Edges: 13786.0
Total Possible Edges: 999000.0
Graph sparsity: 0.9862002002002002
Graph density: 0.013799799799799799
Number of Variables: 13786
Number of Constraints: 14786


In [24]:
# Solve the problem in a standard way using CVXPY
start_time = time.time()
result = problem.solve(solver=cp.CLARABEL)
end_time = time.time()
solve_time = end_time - start_time
obj_val = problem.value
problem_status = problem.status
num_iters = problem.solver_stats.num_iters
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")
print(f"Number of iterations: {num_iters}")
print(f"Status: {problem_status}")

Objective value: 117.00000004121136
Time taken: 0.3505 seconds
Number of iterations: 9
Status: optimal


In [35]:
# Get conic problem form so we can (i) solve standard conic form and (ii) solve using ZAP
cone_params, data, cones = get_standard_conic_problem(problem, solver=cp.CLARABEL)

In [ ]:
# Solve the conic form using CLARABEL
A_cl = cone_params['A']
b_cl = cone_params['b']
q_cl = cone_params['c']
P_cl = sp.csc_matrix(np.zeros((A_cl.shape[1], A_cl.shape[1])))
cones_cl = [clarabel.ZeroConeT(cones['z']), clarabel.NonnegativeConeT(cones['l'])]
settings = clarabel.DefaultSettings()
settings.verbose=False
settings.max_iter = 5000
start_time = time.time()
solver = clarabel.DefaultSolver(P_cl,q_cl,A_cl,b_cl,cones_cl, settings)
end_time = time.time()
soln = solver.solve()
solve_time = end_time - start_time
obj_val = soln.obj_val
problem_status = soln.status
num_iters = soln.iterations
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")
print(f"Number of iterations: {num_iters}")
print(f"Status: {problem_status}")

Objective value: -117.00000004121136
Time taken: 0.0132 seconds
Number of iterations: 9
Status: Solved


In [59]:
# Solve the conic form using ZAP
machine = 'cpu'
dtype = torch.float32
cone_bridge = ConeBridge(cone_params)
cone_admm_devices = [d.torchify(machine=machine, dtype=dtype) for d in cone_bridge.devices]
cone_admm = ADMMSolver(
    machine=machine,
    dtype=dtype,
    atol=1e-3,
    rtol=1e-3,
    num_iterations=3000,
)
start_time = time.time()
cone_solution_admm, cone_history_admm = cone_admm.solve(
    net=cone_bridge.net, devices=cone_admm_devices, time_horizon=cone_bridge.time_horizon
)
end_time = time.time()
solve_time = end_time - start_time
obj_val = cone_solution_admm.objective
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")



ADMM converged in 1479 iterations.
Objective value: -118.25161743164062
Time taken: 1.0617 seconds
